# Get MAEs for comparison

This notebook loads different models with their correct configs one by one and calculates predictions and mean average errors for different speeds

## Setup
- Download and unpack training data
- Create DonkeyCar project and load configuration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('donkeycar')
sys.path.append('src')

In [ ]:
import os
# do we have the correct env loaded?
assert 'donkey' == os.environ['CONDA_DEFAULT_ENV'], os.environ['CONDA_DEFAULT_ENV']

In [6]:
![ ! -d "data/" ] && \
mkdir data && \
curl -L "https://www.dropbox.com/s/eltelvjt12a4ny1/data.zip?dl=1" --output data/data.zip && \
unzip data/data.zip -d data

In [7]:
![ ! -d "models/" ] && \
mkdir models && \
curl -L "https://www.dropbox.com/s/k9amvpqz3lcu7f3/models.zip?dl=1" --output models/models.zip && \
unzip models/models.zip -d models

In [8]:
![ ! -d "mycar/" ] && \
donkey createcar --path mycar && \
cp config/myconfig.py mycar/

In [35]:
from dataloader import load_data
import donkeycar as dk
cfg = dk.load_config(config_path='mycar/config.py')

# None means all the data is used.
# use a smaller size, like 200 for testing end to end.
TUBRECORD_SIZE = None

cfg.SEQUENCE_LENGTH = 3
cfg.WANDB_ENABLED = False
cfg.TRANSFORMATIONS = ['CROP']
cfg.ROI_CROP_TOP = 60
cfg.ROI_CROP_KEEP_SIZE = False


N_FOLDS = None
DATA_PATH = 'data'
MODELS_PATH = 'models'

model_type = '3d-modified-steering'


loading config file: mycar/config.py
loading personal config over-rides from myconfig.py


In [36]:

import itertools

DIRECTION = 'CC'

tub_records_80_speed, tub_records_85_speed, tub_records_90_speed = load_data(cfg, DATA_PATH, DIRECTION, TUBRECORD_SIZE)

all_80_speed_data = list(itertools.chain(*list(tub_records_80_speed.values())))
all_85_speed_data = list(itertools.chain(*list(tub_records_85_speed.values())))
all_90_speed_data = list(itertools.chain(*list(tub_records_90_speed.values())))

INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-1-CC-80/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-1-CC-80']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-1-CC-80/catalog_5.catalog
Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-3-CC-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-3-CC-85']
INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-3-CC-85']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-3-CC-85/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/1-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/1-2-CC-90/catalog_4.catalog


INFO:donkeycar.pipeline.types:Loading tubs from paths ['data/4-2-CC-90']


Using catalog /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/data/4-2-CC-90/catalog_7.catalog


In [37]:
from donkeycar.utils import get_model_by_type
model = get_model_by_type(model_type, cfg)
model_path = '/Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/models/threeForSunday/3d-CC-only90.h5'
model.load(model_path)

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3
INFO:donkeycar.parts.keras:Loading model /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/models/threeForSunday/3d-CC-only90.h5
INFO:donkeycar.parts.interpreter:Loading model /Users/kristjan.roosild/PycharmProjects/dat-sci-master-thesis/models/threeForSunday/3d-CC-only90.h5


In [38]:
from losses import Results
results_90_speed = []
results_85_speed = []
results_80_speed = []

for tub_name, records in tub_records_90_speed.items():
    tub_results_holder = Results(tub_name, DIRECTION, cfg, 1, model, records, is_train=True)
    print(tub_results_holder)
    results_90_speed.append(tub_results_holder)

for tub_name, records in tub_records_85_speed.items():
    tub_results_holder = Results(tub_name, DIRECTION, cfg, 1, model, records, is_train=True)
    print(tub_results_holder)
    results_85_speed.append(tub_results_holder)

for tub_name, records in tub_records_80_speed.items():
    tub_results_holder = Results(tub_name, DIRECTION, cfg, 1, model, records, is_train=True)
    print(tub_results_holder)
    results_80_speed.append(tub_results_holder)




seq size 3
len ground_truth 4542
36/36 [==============================] - 19s 528ms/step
predictions shape (4542,)
1-2-CC-90 has 4542 predictions with an mse of 0.1094225182002424
seq size 3
len ground_truth 5963
47/47 [==============================] - 24s 513ms/step
predictions shape (5963,)
4-2-CC-90 has 5963 predictions with an mse of 0.10485559264303558
seq size 3
len ground_truth 4272
34/34 [==============================] - 18s 515ms/step
predictions shape (4272,)
1-3-CC-85 has 4272 predictions with an mse of 0.12701774859658152
seq size 3
len ground_truth 4398
35/35 [==============================] - 15s 430ms/step
predictions shape (4398,)
4-3-CC-85 has 4398 predictions with an mse of 0.0779713023654179
seq size 3
len ground_truth 2998
24/24 [==============================] - 14s 564ms/step
predictions shape (2998,)
1-1-CC-80 has 2998 predictions with an mse of 0.12470184548654782
seq size 3
len ground_truth 4705
37/37 [==============================] - 22s 606ms/step
predicti

In [49]:
str.format("{:.2f}", 0.0001)

'0.00'

In [39]:
from collections import defaultdict
CC_results = defaultdict(dict)

CC_results[1]['0.8'] = results_80_speed
CC_results[1]['0.85'] = results_85_speed
CC_results[1]['0.9'] = results_90_speed


In [40]:
from donkeycar.utils import get_model_by_type

get_model_by_type(model_type, cfg).interpreter.model.summary()

INFO:donkeycar.utils:get_model_by_type: model type is: 3d-modified-steering
INFO:donkeycar.parts.keras:input_shape (60, 160, 3) num_outputs 1
INFO:donkeycar.parts.keras:Created Keras3D_CNN_ModifiedOnlySteering with interpreter: KerasInterpreter
INFO:donkeycar.parts.keras:Num outputs 1
INFO:donkeycar.parts.keras:Sequence length 3


Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_in (InputLayer)          [(None, 3, 60, 160, 3)]   0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 1, 20, 53, 16)     1312      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 10, 26, 16)     0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 1, 10, 26, 32)     4640      
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 1, 5, 13, 32)      0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 1, 5, 13, 32)      9248      
_________________________________________________________________
flatten_1 (Flatten)          (None, 2080)              0     

In [42]:

results_unpacked = []

for fold, fold_results in CC_results.items():
    for speed_name, results in fold_results.items():
        for r in results:
            for pred, gt in zip(r.predictions, r.ground_truths):
                results_unpacked.append([r.name, r.direction, r.speed, r.fold, r.is_train, pred, gt])

len(results_unpacked)

26878

In [43]:
import pandas as pd
df = pd.DataFrame(results_unpacked, columns=['name', 'direction', 'speed', 'fold', 'is_train', 'predicted_angle', 'ground_truth_angle'])


In [44]:
df['absolute_error'] = abs(df.predicted_angle - df.ground_truth_angle)

In [45]:
df.groupby(['speed']).mean()

,fold,is_train,predicted_angle,ground_truth_angle,absolute_error
speed,,,,,
0.80,1,True,-0.299867,-0.217350,0.240558
0.85,1,True,-0.304577,-0.223128,0.234309
0.90,1,True,-0.285910,-0.264934,0.246377


In [17]:
# df.to_csv('ss', index=False)